In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import re, json

tokenizer = AutoTokenizer.from_pretrained("mofawzy/gpt-2-negative-reviews")
model = AutoModelForCausalLM.from_pretrained("mofawzy/gpt-2-negative-reviews")

In [2]:
MODEL_PATH = "./gpt-2-negative-reviews/"

nlp = pipeline(
    "sentiment-analysis",
    model=MODEL_PATH,
    tokenizer=MODEL_PATH,
)

Some weights of the model checkpoint at ./gpt-2-negative-reviews/ were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at ./gpt-2-negative-reviews/ and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
with open('init.txt') as f:
    lines = [line.rstrip() for line in f]
f.close()


MAX_LEN=25
MIN_LEN=5
TOP_K=5_000
N_SEQ=5_00
results = []

for keyword in lines:
    input_ids = tokenizer.encode(keyword, return_tensors='pt')
    sample_outputs = model.generate(None,
                                       do_sample=True, 
                                       max_length=MAX_LEN, 
                                       min_length=MIN_LEN,
                                       top_k=TOP_K,
                                       num_return_sequences=N_SEQ)

    for sample_output in sample_outputs:
        gen = tokenizer.decode(sample_output)
        result = nlp(gen)
        if result[0]['score'] >= 0.70:
            with open('data/train_over_sampling.json', encoding='utf-8', mode='a+') as output:
                res = re.sub(r'[^\w\s]', '', gen)
                row={
                        "review": res.replace("\n", " "),
                        "label": str(0)
                }
                output.write(json.dumps(row))
                output.write("\n")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [ ]:
import re 
with open('generated.txt', 'w') as f:
    for item in results:
        res = re.sub(r'[^\w\s]', '', item)
        print(res)
        print('========')
        print(item)
        break
        f.write(res)

In [ ]:
keywords = ['سئ', 'وحش', 'ممل', 'مقرف']
for keyword in keywords:
    input_ids = tokenizer.encode(keyword, return_tensors='pt')
    max_length = 40
    sample_outputs = model.generate(None,
                                       do_sample=True, 
                                       max_length=max_length, 
                                       min_length=10,
                                       top_k=30,
                                       num_return_sequences=5)

    for i, sample_output in enumerate(sample_outputs):
        gen = tokenizer.decode(sample_output.tolist())
        print(20*'=', '\n')
        print(f"Sample {i+1}")
        for line in gen.split('.')[:-1]:
            print(line)

### Eval
Fine-tune your LM on a downstream task

In [ ]:
from transformers import TokenClassificationPipeline, pipeline


MODEL_PATH = "./gpt2-arabic-sentence-generator/"

nlp = pipeline(
    "sentiment-analysis",
    model=MODEL_PATH,
    tokenizer=MODEL_PATH,
)


In [ ]:
result = nlp("كتاب وحش اوي")

In [ ]:
result[0]['score']

### New Model Fine-Tuning

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import re
import json

In [11]:
tokenizer = AutoTokenizer.from_pretrained("mofawzy/gpt-2-medium-ar")
model = AutoModelForCausalLM.from_pretrained("mofawzy/gpt-2-medium-ar")

In [ ]:
with open('init.txt') as f:
    lines = [line.rstrip() for line in f]
f.close()


MAX_LEN=150
MIN_LEN=25
TOP_K=5_00
N_SEQ=5_00
results = []

for keyword in lines:
    input_ids = tokenizer.encode(keyword, return_tensors='pt')
    sample_outputs = model.generate( 
                                    input_ids, 
                                    max_length=MAX_LEN,  
                                    num_return_sequences=N_SEQ,
                                    no_repeat_ngram_size=2,
                                    repetition_penalty=1.5,
                                    top_p=0.92,
                                    temperature=.85,
                                    do_sample=True,
                                    top_k=TOP_K,
                                    early_stopping=True)

    for i, sample_output in enumerate(sample_outputs):
        gen = tokenizer.decode(sample_output, skip_special_tokens=True)
        with open('data/train_over_sampling_2.json', encoding='utf-8', mode='a+') as output:
            res = re.sub(r'[^\w\s]', '', gen)
            row={
                "review": res.replace("\n", " "),
                "label": str(0)
            }
            output.write(json.dumps(row))
            output.write("\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
